In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [ ]:
  def get_url(position):
    "generate a url based on the postion passed"
    template = 'https://merojob.com/search/?q={}'
    url=template.format(position)
    return url

In [ ]:
def get_record(card):
  """Extract the job data from a single record"""
  a_tag=card.h1.a
  
  job_title=a_tag.get('title') 
  # print(job_title)
  job_url='https://www.merojob.com'+a_tag.get('href')
  # print(job_url)
  a_tag2=card.h3.a
  h_tag=card.h3
  company=a_tag2.text.strip()
  # print(company)
  company_postings='https://merojob.com'+a_tag2.get('href')
  # print(company_postings)
  location_element = card.find('span',itemprop='addressLocality')
  job_location = location_element.text.strip()
  # print(job_location)
  skills=card.find('span',itemprop='skills').text
  skills=skills.replace('\n',' ')
  # print(skills)
  apply_before=card.find('span', class_='icon-time').next_sibling.next_sibling.text
  apply_before=apply_before.replace('\xa0',' ')
  today=datetime.today().strftime('%Y-%m-%d')
  # print(today)
  try:

    views=card.find('span', class_='icon').next_sibling.strip()
    # print(views)
  except AttributeError:
    views='0'
  record=(job_title,company,job_url,company_postings,job_location,skills,apply_before,today)
   
    
  return record

In [ ]:
def main(position):
  records = []
  url=get_url('python')

  while True:
      try:
          response = requests.get(url)
          soup = BeautifulSoup(response.text, 'html.parser')
          
          cards = soup.find_all('div', itemtype='http://schema.org/JobPosting')
          for card in cards:
              record = get_record(card)
              records.append(record)

          next_page_link = soup.find('a', {'class': 'pagination-next'})
          if next_page_link is None:
              break

          url = 'https://merojob.com' + next_page_link.get('href')

      except Exception as e:
          print(f"An error occurred: {e}")
          break
  with open('results.csv','w',newline='', encoding='utf-8') as f:
    writer=csv.writer(f)
    writer.writerow(['job_title','company','job_url','company_postings','job_location','skills','apply_before','today'])
    writer.writerows(records)



In [ ]:
# records
main('python')


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('results.csv')

In [ ]:
df.head(1)

,job_title,company,job_url,company_postings,job_location,skills,apply_before,today
0,Data Engineering Manager,Cedar Gate Services,https://www.merojob.com/data-engineering-manager/,https://merojob.com/employer/cedar-gate-servic...,"Lalitpur, Nepal",Interpersonal Skills Communication Sql Python...,Apply Before: 2 weeks from now,2023-05-25


In [ ]:
df.to_json('results1.json',orient='records')

In [ ]:
import json

# Open the JSON file
with open('results1.json', 'r') as file:
    # Load the JSON data
    data = json.load(file)





In [ ]:
sth = requests.get(data[0]['job_url'])

In [ ]:
sth.reason

'OK'